In [1]:
import pandas as pd
import re
from datetime import datetime
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings

/Users/stephenahiabah/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
df_rankings = pd.read_csv("DataIn/fifa_ranking-2024-04-04.csv")
df_results = pd.read_csv("DataIn/Results.csv")

In [3]:
df_results["date"] = pd.to_datetime(df_results["date"])
euro_games = df_results[(df_results.date.dt.year == 2024) & (df_results.tournament == "UEFA Euro")]
df_results.dropna(inplace=True)
df_results = df_results[(df_results["date"] >= "2019-8-1")].reset_index(drop=True)

In [4]:
df_rankings.rank_date = df_rankings.rank_date.apply(lambda x: x.replace("2024-04-04", datetime.today().strftime('%Y-%m-%d')))
df_rankings["rank_date"] = pd.to_datetime(df_rankings["rank_date"])
df_rankings = df_rankings[(df_rankings["rank_date"] >= "2019-8-1")].reset_index(drop=True)
df_rankings["country_full"] = df_rankings["country_full"].str.replace("Czechia", "Czech Republic").str.replace("IR Iran", "Iran").str.replace("Korea Republic", "South Korea").str.replace("USA", "United States")

df_rankings = df_rankings.set_index(['rank_date']).groupby(['country_full'], group_keys=False).resample('D').first().fillna(method='ffill').reset_index()

In [5]:
df_results

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,2019-08-02,Iraq,Palestine,2.0,1.0,WAFF Championship,Karbala,Iraq,False
1,2019-08-02,Lebanon,Syria,2.0,1.0,WAFF Championship,Karbala,Iraq,True
2,2019-08-04,Jordan,Bahrain,0.0,1.0,WAFF Championship,Erbil,Iraq,True
3,2019-08-04,Saudi Arabia,Kuwait,1.0,2.0,WAFF Championship,Erbil,Iraq,True
4,2019-08-05,Syria,Yemen,1.0,1.0,WAFF Championship,Karbala,Iraq,True
...,...,...,...,...,...,...,...,...,...
4559,2024-06-11,Saint Kitts and Nevis,Bahamas,1.0,0.0,FIFA World Cup qualification,Basseterre,Saint Kitts and Nevis,False
4560,2024-06-11,Saint Lucia,Aruba,2.0,2.0,FIFA World Cup qualification,Bridgetown,Barbados,True
4561,2024-06-11,Guyana,Belize,3.0,1.0,FIFA World Cup qualification,Bridgetown,Barbados,True
4562,2024-06-11,Dominican Republic,British Virgin Islands,4.0,0.0,FIFA World Cup qualification,San Cristóbal,Dominican Republic,False


In [6]:
df_ranked = df_results.merge(df_rankings[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], left_on=["date", "home_team"], right_on=["rank_date", "country_full"]).drop(["rank_date", "country_full"], axis=1)

df_ranked = df_ranked.merge(df_rankings[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], left_on=["date", "away_team"], right_on=["rank_date", "country_full"], suffixes=("_home", "_away")).drop(["rank_date", "country_full"], axis=1)

In [7]:
df_ranked.to_csv("data/data_rank.csv", index=False)


In [18]:
df_ranked

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,total_points_home,previous_points_home,rank_home,rank_change_home,total_points_away,previous_points_away,rank_away,rank_change_away
0,2019-09-29,Bangladesh,Bhutan,4.0,1.0,Friendly,Dhaka,Bangladesh,False,912.00,922.00,187.0,5.0,916.00,916.00,185.0,-1.0
1,2019-09-30,Botswana,Liberia,0.0,0.0,Friendly,Gaborone,Botswana,False,1059.00,1069.00,148.0,1.0,1053.00,1044.00,152.0,0.0
2,2019-10-02,Mexico,Trinidad and Tobago,2.0,0.0,Friendly,Toluca,Mexico,False,1603.00,1604.00,12.0,0.0,1226.00,1226.00,100.0,-1.0
3,2019-10-03,Bangladesh,Bhutan,2.0,0.0,Friendly,Dhaka,Bangladesh,False,912.00,922.00,187.0,5.0,916.00,916.00,185.0,-1.0
4,2019-10-05,Jordan,Singapore,0.0,0.0,Friendly,Amman,Jordan,False,1235.00,1229.00,98.0,-1.0,1019.00,999.00,157.0,-5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3808,2024-06-11,Australia,Palestine,5.0,0.0,FIFA World Cup qualification,Perth,Australia,False,1554.82,1539.22,23.0,-2.0,1227.20,1217.60,97.0,-2.0
3809,2024-06-11,Cuba,Cayman Islands,3.0,0.0,FIFA World Cup qualification,Santiago,Cuba,False,980.65,980.65,169.0,0.0,851.19,851.19,197.0,0.0
3810,2024-06-11,Guyana,Belize,3.0,1.0,FIFA World Cup qualification,Bridgetown,Barbados,True,1018.14,1018.14,157.0,0.0,918.68,918.68,182.0,0.0
3811,2024-06-11,Dominican Republic,British Virgin Islands,4.0,0.0,FIFA World Cup qualification,San Cristóbal,Dominican Republic,False,1040.77,1040.77,150.0,-1.0,807.57,807.57,207.0,0.0


In [8]:
df_results = pd.read_csv("DataIn/Results.csv")
df_results["date"] = pd.to_datetime(df_results["date"])
euro_games = df_results[(df_results.date.dt.year == 2024) & (df_results.tournament == "UEFA Euro")]
euro_games.to_csv("data/euro_matches.csv", index=False)

In [9]:
top5_leagues = 'https://fbref.com/en/comps/Big5/Big-5-European-Leagues-Stats'

In [16]:
def create_full_stats_db(top5_leagues):
    shot_ = top5_leagues
    page =requests.get(shot_)
    soup = BeautifulSoup(page.content, 'html.parser')
    html_content = requests.get(shot_).text.replace('<!--', '').replace('-->', '')
    shot_df = pd.read_html(html_content)
    # shot_df[0].columns = shot_df[0].columns.droplevel(0) # drop top header row
    stats = shot_df[0]
    stats = stats[stats['Squad'] != 'Squad']    

    return stats

In [17]:
create_full_stats_db(top5_leagues)

,Rk,Squad,Country,LgRk,MP,W,D,L,GF,GA,...,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper
0,1,Paris S-G,fr FRA,1,16,12,4,0,44,14,...,40,2.50,41.7,15.7,26.0,1.63,W D D W W,47180,Bradley Barcola - 10,Gianluigi Donnarumma
1,2,Liverpool,eng ENG,1,17,13,3,1,40,17,...,42,2.47,38.2,16.3,21.9,1.29,W D D W W,60277,Mohamed Salah - 16,Alisson
2,3,Bayern Munich,de GER,1,15,11,3,1,47,13,...,36,2.40,34.5,9.6,25.0,1.67,W D W L W,75000,Harry Kane - 14,Manuel Neuer
3,4,Inter,it ITA,2,17,12,4,1,45,15,...,40,2.35,33.3,14.9,18.4,1.08,D W W W W,72304,Marcus Thuram - 12,Yann Sommer
4,5,Atalanta,it ITA,1,18,13,2,3,43,20,...,41,2.28,33.5,16.7,16.9,0.94,W W W W W,22704,Mateo Retegui - 12,Marco Carnesecchi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,92,Montpellier,fr FRA,18,15,2,3,10,15,38,...,9,0.60,17.0,29.5,-12.5,-0.84,W L D L D,13876,Arnaud Nordin - 4,Benjamin Lecomte
92,93,Monza,it ITA,20,18,1,7,10,16,25,...,10,0.56,15.8,22.5,-6.7,-0.37,D D L L L,12288,"Milan Đurić, Dany Mota - 4",Stefano Turati
93,94,Holstein Kiel,de GER,17,15,2,2,11,19,38,...,8,0.53,16.2,27.6,-11.4,-0.76,L L L L W,14874,Shuto Machino - 6,Timon Weiner
94,95,Bochum,de GER,18,15,1,3,11,13,35,...,6,0.40,16.7,32.1,-15.4,-1.03,L L L D W,25565,Matúš Bero - 3,Patrick Drewes
